In [1]:
import sys

import igris_c_sdk as igc_sdk

In [2]:
domain_id = 0

channel_instance = igc_sdk.ChannelFactory.Instance()
channel_instance.Init(domain_id)

client = igc_sdk.IgrisC_Client()
client.Init()
client.SetTimeout(20000)


[ChannelFactory] Initialized on domain 0
[Publisher] Initialized topic: rt/service/bms_init/request
[Publisher] Initialized topic: rt/service/torque/request
[Publisher] Initialized topic: rt/service/control_mode/request
[Subscriber] Initialized topic: rt/service/bms_init/response
[Subscriber] Initialized topic: rt/service/torque/response
[Subscriber] Initialized topic: rt/service/control_mode/response
[IgrisC_Client] Service API initialized


In [ ]:

lowstate_subscriber = igc_sdk.LowStateSubscriber("rt/lowstate")
lowstate_cnt = 0

In [ ]:
def lowstate_callback(msg):
    imu_state = msg.imu_state()
    quaternion = imu_state.quaternion()
    gyroscope = imu_state.gyroscope()
    accelerometer = imu_state.accelerometer()
    rpy = imu_state.rpy()

    motor_states = msg.motor_state()
    motor_1 = motor_states[0]
    motor_31 = motor_states[30]

    q = motor_1.q()
    dq = motor_1.dq()
    tau_est = motor_1.tau_est()
    temp = motor_1.temperature()
    status = motor_1.status_bits()

    joint_states = msg.joint_state()
    joint_1 = joint_states[0]
    joint_31 = joint_states[30]
    q = joint_1.q()
    dq = joint_1.dq()
    tau_est = joint_1.tau_est()
    status_bits = joint_1.status_bits()

    print(f"motor 1 pos{q}")

In [ ]:
lowstate_subscriber.init(lowstate_callback)

In [ ]:
del lowstate_subscriber

In [ ]:
lowcmd_publisher = igc_sdk.LowCmdPublisher("rt/lowcmd")
lowcmd_publisher.init()

In [ ]:
def call_init_bms(
    client: igc_sdk.IgrisC_Client, bms_init_type: igc_sdk.BmsInitType, type_name: str
):
    print(f"Calling Init with {type_name}...")
    res = client.InitBms(bms_init_type, 20000)

    result = "SUCCESS" if res.success() else "FAILED" + " - " + res.message()
    print(f"Init {type_name} result: {result}")


def call_set_torque(client: igc_sdk.IgrisC_Client, torque_type: igc_sdk.TorqueType, type_name: str):
    print(f"Calling Set Torque with {type_name}...")
    res = client.SetTorque(torque_type, 20000)

    result = "SUCCESS" if res.success() else "FAILED" + " - " + res.message()
    print(f"Set Torque {type_name} result: {result}")


def call_set_control_mode(
    client: igc_sdk.IgrisC_Client, control_mode: igc_sdk.ControlMode, mode_name: str
):
    print(f"Calling Set Control Mode with {mode_name}...")
    res = client.SetControlMode(control_mode, 5000)

    result = "SUCCESS" if res.success() else "FAILED" + " - " + res.message()
    print(f"Set Control Mode {mode_name} result: {result}")

In [5]:
call_init_bms(client, igc_sdk.BmsInitType.BMS_INIT, "BMS_INIT") # bms 키기

Calling Init with BMS_INIT...
Init BMS_INIT result: SUCCESS


In [4]:
call_init_bms(client, igc_sdk.BmsInitType.BMS_INIT_NONE, "BMS_INIT_NONE") # bms 끄기

Calling Init with BMS_INIT_NONE...
Init BMS_INIT_NONE result: SUCCESS


In [ ]:
call_init_bms(client, igc_sdk.BmsInitType.MOTOR_INIT, "MOTOR_INIT") # 모터 초기화

Calling Init with MOTOR_INIT...
Init MOTOR_INIT result: FAILED - Request timeout


1764907776.785689 [0]        tev: ddsi_udp_conn_write to udp/192.168.0.65:7424 failed with retcode -1
1764907776.785767 [0]        tev: ddsi_udp_conn_write to udp/192.168.0.65:7436 failed with retcode -1


In [ ]:
call_init_bms(client, igc_sdk.BmsInitType.BMS_AND_MOTOR_INIT, "BMS_AND_MOTOR_INIT") # bms 키고 모터 초기화

In [ ]:
call_set_torque(client, igc_sdk.TorqueType.TORQUE_OFF, "TORQUE_OFF") # 토크 오프

In [ ]:
call_set_torque(client, igc_sdk.TorqueType.TORQUE_ON, "TORQUE_ON") # 토크 온

In [ ]:
call_set_control_mode(client, igc_sdk.ControlMode.CONTROL_MODE_LOW_LEVEL, "CONTROL_MODE_LOW_LEVEL") # 이걸 해야지만 로우레벨 컨트롤 됨

In [ ]:
# call_set_control_mode(
#     client, igc_sdk.ControlMode.CONTROL_MODE_HIGH_LEVEL, "CONTROL_MODE_HIGH_LEVEL"
# )

In [ ]:
low_cmd_msg = igc_sdk.LowCmd()
for i in range(0, 31):
    if i == 30:
        motor_cmd = low_cmd_msg.motors()[i]
        motor_cmd.q(0.0)
        motor_cmd.dq(0.0)
        motor_cmd.tau(0.0)
        motor_cmd.kp(5)
        motor_cmd.kd(0.1)
        motor_cmd.id(i)
    elif i == 29:
        motor_cmd = low_cmd_msg.motors()[i]
        motor_cmd.q(0)
        motor_cmd.dq(0.0)
        motor_cmd.tau(0.0)
        motor_cmd.kp(2)
        motor_cmd.kd(0.05)
        motor_cmd.id(i)
    else:
        motor_cmd = low_cmd_msg.motors()[i]
        motor_cmd.q(0.0)
        motor_cmd.dq(0.0)
        motor_cmd.tau(0.0)
        motor_cmd.kp(0.0)
        motor_cmd.kd(0.0)
        motor_cmd.id(i)
low_cmd_msg.kinematic_mode(igc_sdk.KinematicMode.MS)
lowcmd_publisher.write(low_cmd_msg)